<a href="https://colab.research.google.com/github/Soikey/Drug-Discovery/blob/main/ensemble_models_for_BACE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Feature Selection

In [1]:
!pip install rdkit deap scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random

from rdkit import Chem
from rdkit.Chem import Descriptors
from deap import base, creator, tools, algorithms

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

Load and Preprocess Molecular Data

In [18]:
data = pd.read_csv("/content/bace1_bioactivity_data_curated.csv")


Getting all RDKit molecular descriptor names and Function to compute molecular descriptors

In [19]:
descriptor_names = [desc[0] for desc in Descriptors.descList]

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0] * len(descriptor_names)  # Handle invalid molecules
    return [Descriptors.__dict__[name](mol) for name in descriptor_names]

In [20]:
data["class"]

,class
0,active
1,active
2,active
3,intermediate
4,intermediate
...,...
7454,intermediate
7455,intermediate
7456,inactive
7457,intermediate


In [21]:
y = data["class"].map({"active": 1, "inactive": 0, "intermediate": 0}).values

X = np.array([compute_descriptors(smiles) for smiles in data["canonical_smiles"]])
X_df = pd.DataFrame(X, columns=descriptor_names)
X_df = X_df.dropna(axis=1, how='any').loc[:, (X_df != X_df.iloc[0]).any()] # Remove NaN and constant-value descriptors

# Convert back to NumPy array
X = X_df.values
num_features = X.shape[1]

print(f"Number of descriptors after cleaning: {num_features}")


Number of descriptors after cleaning: 191


In [22]:
print("NaN in X:", np.isnan(X).sum())  # Count NaNs
print("Inf in X:", np.isinf(X).sum())  # Count Infs
print("Max value in X:", np.max(X))  # Identify largest value
print("Min value in X:", np.min(X))  # Identify smallest value


NaN in X: 0
Inf in X: 0
Max value in X: 8.127863505038417e+84
Min value in X: -39.298382243297695


max value (8.127863505038417e+84) is extremely large, which is likely causing the **overflow issue** in Scikit-learn. This is far beyond what float32 or even float64 can handle in practical ML models

In [23]:
clip_min, clip_max = -1e3, 1e3


num_values_clipped = np.sum((X < clip_min) | (X > clip_max))
num_rows_clipped = np.sum(np.any((X < clip_min) | (X > clip_max), axis=1))

print(f"Total number of values clipped: {num_values_clipped}")
print(f"Total number of rows affected by clipping: {num_rows_clipped}")

# # Apply clipping
# X = np.clip(X, clip_min, clip_max)

# # Debugging: Check again
# print("Post-processing: Max value in X:", np.max(X))  # Should be <= 1e3
# print("Post-processing: Min value in X:", np.min(X))

Total number of values clipped: 12225
Total number of rows affected by clipping: 7437


In [24]:
# Count how many values were clipped per descriptor
descriptor_clip_counts = np.sum((X < clip_min) | (X > clip_max), axis=0)

# Find the descriptors that are most affected
problematic_descriptors = [(descriptor_names[i], descriptor_clip_counts[i]) for i in range(len(descriptor_names))]
problematic_descriptors.sort(key=lambda x: x[1], reverse=True)  # Sort by most affected

# Print the top 10 most problematic descriptors
print("Top 10 most extreme descriptors:")
for desc, count in problematic_descriptors[:10]:
    print(f"{desc}: {count} values clipped")

IndexError: index 191 is out of bounds for axis 0 with size 191

### Define Genetic Algorithm Setup  


In [9]:
# Genetic Algorithm Parameters
population_size = 50
generations = 200
crossover_prob = 0.9
mutation_prob = 0.05
stagnation_limit = 10  # Stop if best subset remains unchanged for 10 generations

# Define GA structure
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Generate initial population (random feature subsets)
def random_subset():
    return [random.randint(0, 1) for _ in range(num_features)]

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, random_subset)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


### Define Fitness Function
Fitness function: Evaluate selected descriptors using Random Forest + Cross-validation



In [11]:
# Fitness function: Evaluate selected descriptors using Random Forest + Cross-validation
def evaluate(individual):
    selected_features = [i for i in range(num_features) if individual[i] == 1]

    # If no features are selected, return worst fitness
    if len(selected_features) == 0:
        return (0,)

    X_selected = X[:, selected_features].astype(np.float64)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(model, X_selected, y, cv=10, scoring='accuracy')

    return (np.mean(scores),)



###  Register Genetic Operators
Define crossover, mutation, and selection methods  

mate:Two parents combine  

nutate: Flip feature selection bits  

select: Tournament selection

In [12]:
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=mutation_prob)
toolbox.register("select", tools.selTournament, tournsize=3)

### Running the Genetic Algorithm

In [27]:
print(np.sum(np.isinf(y)))

0


In [13]:
def run_ga():
    population = toolbox.population(n=population_size)
    best_fitness = 0
    stagnation_counter = 0

    for gen in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob)
        fits = list(map(toolbox.evaluate, offspring))

        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit

        population = toolbox.select(offspring, k=len(population))
        current_best = max(population, key=lambda ind: ind.fitness.values[0])

        # Stagnation Check
        if current_best.fitness.values[0] > best_fitness:
            best_fitness = current_best.fitness.values[0]
            stagnation_counter = 0
        else:
            stagnation_counter += 1

        if stagnation_counter >= stagnation_limit:
            print(f"GA converged after {gen+1} generations.")
            break

    best_subset = max(population, key=lambda ind: ind.fitness.values[0])
    selected_features = [descriptor_names[i] for i in range(num_features) if best_subset[i] == 1]

    print(f"Best accuracy: {best_fitness:.4f}")
    print(f"Selected descriptors: {selected_features}")

    return selected_features

# Run GA feature selection
selected_descriptors = run_ga()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py", line 375, in fit
    estimator._compute_missing_values_in_feature_mask(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/tree/_classes.py", line 222, in _compute_missing_values_in_feature_mask
    _assert_al

 Filter the dataset with selected descriptors

In [ ]:
X_selected = X_df[selected_descriptors].values


## 2D PCA Visualization of Chemical *Space*

In [ ]:
# Apply PCA for dimensionality reduction to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_selected)

# Convert to DataFrame
pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df["Activity"] = y

# Plot chemical space using PCA
plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="Activity", palette=["red", "blue"], alpha=0.7)
plt.title("Chemical Space Visualization using PCA (Selected Descriptors)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Activity", labels=["Inactive", "Active"])
plt.show()


2D t-SNE Visualization of Chemical Space

In [ ]:
# Apply t-SNE for non-linear dimensionality reduction
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_selected)

# Convert to DataFrame
tsne_df = pd.DataFrame(X_tsne, columns=["t-SNE1", "t-SNE2"])
tsne_df["Activity"] = y

# Plot chemical space using t-SNE
plt.figure(figsize=(8, 6))
sns.scatterplot(data=tsne_df, x="t-SNE1", y="t-SNE2", hue="Activity", palette=["red", "blue"], alpha=0.7)
plt.title("Chemical Space Visualization using t-SNE (Selected Descriptors)")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.legend(title="Activity", labels=["Inactive", "Active"])
plt.show()


## Train and Evaluate Ensemble Models

In [29]:
# Filter dataset with selected descriptors
X_selected = X_df[selected_descriptors].values

# Split data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ensemble models
models = {
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Make predictions
    accuracy = accuracy_score(y_test, y_pred)  # Compute accuracy
    print(f"{name} Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: Input X contains infinity or a value too large for dtype('float32').

### Cross-Validation for More Reliable Evaluation

In [ ]:
# Perform 10-fold cross-validation on all models
for name, model in models.items():
    scores = cross_val_score(model, X_selected, y, cv=10, scoring='accuracy')
    print(f"{name} Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")
